# 1.Mango Classification
This notebook demonstrates how to build an ensemble model using EfficientNetB0, VGG16, and ResNet50 for mango classification

In [ ]:
# Import necessary libraries
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, VGG16, ResNet50
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_pre
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_pre
from tensorflow.keras.applications.resnet50 import preprocess_input as res_pre
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, BatchNormalization, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import random
import math
import cv2

## 2. Check GPU Availability
Verify if GPU is available for faster training

In [ ]:
# Check GPU availability
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# Setup distribution strategy (only if GPUs are available)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    strategy = tf.distribute.MirroredStrategy()
    print(" Strategy initialized for GPU")
    print("Number of replicas:", strategy.num_replicas_in_sync)
else:
    strategy = tf.distribute.get_strategy()
    print(" Using default strategy for CPU")

# 3. Data Loading and Preparation
Load and preprocess the mango dataset

In [ ]:
# Dataset configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Dataset paths
base_path = '/kaggle/input/mango7200-12/MangoMerged'
train_dir = os.path.join(base_path, 'train')
val_dir = os.path.join(base_path, 'val')
test_dir = os.path.join(base_path, 'test')

# Get class names
class_names = sorted([
    d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))
])
print("All Name of Classes:", class_names)
NUM_CLASSES = len(class_names)

# Load datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
).prefetch(AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
).prefetch(AUTOTUNE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
).prefetch(AUTOTUNE)

## **4.Visualize few Images from a TensorFlow Dataset**

In [ ]:

def visualize_images_to_pdf(dataset, class_names, num_images=15, pdf_path="visualized_images.pdf"):
    with PdfPages(pdf_path) as pdf:
        plt.figure(figsize=(15, 10))

        # Take just one batch
        for images, labels in dataset.take(1):
            for i in range(min(num_images, images.shape[0])):
                ax = plt.subplot(3, 5, i + 1)

                # Convert image for plotting
                img = images[i].numpy().astype("uint8")
                plt.imshow(img)

                # Convert one-hot label to index
                label_index = tf.argmax(labels[i]).numpy()
                plt.title(class_names[label_index])

                plt.axis("off")

        plt.tight_layout()
        pdf.savefig()
        plt.show()
        plt.close()

    print(f"📄 Visualization saved as {pdf_path}")

visualize_images_to_pdf(train_ds, class_names, num_images=15)


# 5. Build the hybrid Model
Create an ensemble of EfficientNetB0, MobileNetV2, and ResNet50

In [ ]:
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

# Preprocess for each model separately
eff_input = Lambda(eff_pre, name="eff_pre")(inputs)
vgg_input = Lambda(vgg_pre, name="vgg_pre")(inputs)
res_input = Lambda(res_pre, name="res_pre")(inputs)

# Instantiate base models
effnet_base = EfficientNetB0(weights="imagenet", include_top=False)
vgg16_base = VGG16(weights="imagenet", include_top=False)
resnet_base = ResNet50(weights="imagenet", include_top=False)

# Freeze base models
for base_model in [effnet_base, vgg16_base, resnet_base]:
    base_model.trainable = False

# Extract features
eff_features = effnet_base(eff_input)
vgg_features = vgg16_base(vgg_input)
res_features = resnet_base(res_input)

feat1 = GlobalAveragePooling2D()(eff_features)
feat2 = GlobalAveragePooling2D()(vgg_features)
feat3 = GlobalAveragePooling2D()(res_features)

# Concatenate features
merged = Concatenate()([feat1, feat2, feat3])

# Classifier head
x = Dense(512, activation="relu")(merged)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(12, activation="softmax")(x)

# Build model
model = Model(inputs, outputs)
model.summary()


# 6. Train the Model
Train the model with callbacks and early stops

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,        
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "best_model.h5",
    monitor="val_loss",
    save_best_only=True,
    mode="min",
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# Compile
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",   # use 'sparse_categorical_crossentropy' if labels are integer
    metrics=["accuracy"]
)

# Train
history = model.fit(
    train_ds,              #training dataset
    validation_data=val_ds, #validation datase
    epochs=30,
    batch_size=32,
    callbacks=[early_stop, checkpoint, reduce_lr]
)


## 7. Evaluate the Model
Evaluate model performance on test data

In [ ]:
# Evaluate model
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nTest Accuracy (Keras evaluate): {test_acc:.4f}")

y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_true, axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

# Compute confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Save confusion matrices in multiple colors
colormaps = {
    "blue": "Blues",
    "red": "Reds",
    "green": "Greens",
    "magenta": "magma",
    "parula": "viridis" 
}

for color_name, cmap_name in colormaps.items():
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap=cmap_name,
        xticklabels=class_names, yticklabels=class_names
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix ({color_name.capitalize()})")
    plt.tight_layout()
    
    filename = f"{color_name}_confusion_matrix.pdf"
    plt.savefig(filename)

    #  Only display Green in notebook
    if color_name == "green":
        plt.show()
    else:
        plt.close()
    
    print(f"Saved {filename}")


## 8. Plot Training History
Visualize training and validation performance

In [ ]:
def plot_training_history(history, model_name="Model", save_as_pdf=True):
    """Plot training history"""
    epochs = len(history.history['accuracy'])
    epoch_list = list(range(1, epochs + 1))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'{model_name} Performance', fontsize=16)
    fig.subplots_adjust(top=0.88, wspace=0.3)
    
    # Accuracy plot
    ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy', marker='o')
    ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy', marker='o')
    ax1.grid(True, linestyle='--', alpha=0.7)
    ax1.set_xticks(np.arange(1, epochs + 1, max(1, epochs // 10)))
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    ax1.legend(loc="best")
    
    # Loss plot
    ax2.plot(epoch_list, history.history['loss'], label='Train Loss', marker='o')
    ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss', marker='o')
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.set_xticks(np.arange(1, epochs + 1, max(1, epochs // 10)))
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    ax2.legend(loc="best")
    
    plt.tight_layout()
    if save_as_pdf:
        fig.savefig(f"{model_name}_training_history.pdf", dpi=300)
    plt.show()

# Generate two separate PDFs
plot_training_history(history, "MangoFusionNet", save_as_pdf=True)

##  9.predict class and confidence

In [ ]:
# Function to predict class and confidence
def predict(model, image):
    img_array = np.expand_dims(image, axis=0)
    preds = model.predict(img_array, verbose=0)[0]
    pred_idx = np.argmax(preds)
    confidence = round(100 * np.max(preds), 2)
    return pred_idx, confidence

# Function to visualize random predictions and save to PDF
def visualize_predictions_to_pdf(model, dataset, class_names, pdf_path="predictions.pdf", num_images=9):
    # Convert dataset to list of (images, labels)
    all_images, all_labels = [], []
    for images, labels in dataset:
        all_images.append(images.numpy())
        all_labels.append(labels.numpy())
    all_images = np.concatenate(all_images)
    all_labels = np.concatenate(all_labels)

    # Choose random indices
    indices = random.sample(range(len(all_images)), num_images)

    with PdfPages(pdf_path) as pdf:
        plt.figure(figsize=(15, 15))

        for i, idx in enumerate(indices):
            ax = plt.subplot(3, 3, i + 1)
            image = all_images[idx]
            label = all_labels[idx]

            plt.imshow(image.astype("uint8"))

            # Predict
            pred_idx, confidence = predict(model, image)
            predicted_class = class_names[pred_idx]

            # Handle one-hot vs integer labels
            if len(label.shape) > 0 and label.shape[0] > 1:  
                actual_idx = np.argmax(label)
            else:
                actual_idx = int(label)
            
            actual_class = class_names[actual_idx]

            plt.title(f"Actual: {actual_class}\nPredicted: {predicted_class}\nConf: {confidence}%", fontsize=10)
            plt.axis("off")

        plt.tight_layout()
        pdf.savefig(dpi=300)
        plt.show()
        plt.close()

    print(f"📄 Random predictions saved to {pdf_path}")

visualize_predictions_to_pdf(model, test_ds, class_names, pdf_path="hybrid_predictions.pdf", num_images=9)

# **10.Grad-CAM Implementation (XAI)**

In [ ]:
# Image preprocessing per model

def get_img_array(img_path, size, model_type="efficientnet"):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    if model_type=="efficientnet":
        array = eff_pre(array)   
    elif model_type=="vgg16":
        array = vgg_pre(array)   
    elif model_type=="resnet50":
        array = res_pre(array)   
    return array

# Grad-CAM heatmap generator

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_output = predictions[:, pred_index]
    grads = tape.gradient(class_output, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Display & save Grad-CAM

def display_and_save_gradcam(img_path, heatmaps, titles, pdf, alpha=0.4):
    original = cv2.imread(img_path)
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    original = cv2.resize(original, (224, 224))
    n = len(heatmaps)
    fig, axes = plt.subplots(n, 2, figsize=(10, 5 * n))  # n rows, 2 cols
    fig.suptitle(os.path.basename(img_path), fontsize=16, y=1.02)
    for i, (heatmap, title) in enumerate(zip(heatmaps, titles)):
        # Original
        axes[i, 0].imshow(original)
        axes[i, 0].set_title("Original")
        axes[i, 0].axis('off')
        # Heatmap overlay
        heatmap_resized = cv2.resize(heatmap, (224, 224))
        heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
        heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)
        superimposed_img = cv2.addWeighted(original, 1 - alpha, heatmap_colored, alpha, 0)
        axes[i, 1].imshow(superimposed_img)
        axes[i, 1].set_title(title)
        axes[i, 1].axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    pdf.savefig(fig)
    plt.close(fig)


# Build Grad-CAM model wrapper

def build_gradcam_model(base_model, last_conv_layer_name, num_classes):
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return model, last_conv_layer_name

# Configuration

IMG_SIZE = (224, 224)
NUM_CLASSES = 12
N_PER_CLASS = 1  # random images per class
base_path = '/kaggle/input/mango7200-12/MangoMerged'

# Load and wrap models

efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=IMG_SIZE+(3,))
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=IMG_SIZE+(3,))
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE+(3,))

for base in [efficientnet_base, vgg_base, resnet_base]:
    base.trainable = False

eff_model, eff_layer = build_gradcam_model(efficientnet_base, "block7a_project_bn", NUM_CLASSES)
vgg_model, vgg_layer = build_gradcam_model(vgg_base, "block5_conv3", NUM_CLASSES)
res_model, res_layer = build_gradcam_model(resnet_base, "conv5_block3_out", NUM_CLASSES)

# Collect images per class

class_to_images = {}
for root, _, files in os.walk(base_path):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            class_name = os.path.basename(root)
            if class_name not in class_to_images:
                class_to_images[class_name] = []
            class_to_images[class_name].append(os.path.join(root, file))

selected_images = {
    cls: random.sample(imgs, min(N_PER_CLASS, len(imgs)))
    for cls, imgs in class_to_images.items()
}

pdf_output = "gradcam_per_class_1_random.pdf"
with PdfPages(pdf_output) as pdf:

    # One page per image
    for cls, img_paths in selected_images.items():
        print(f"Processing class '{cls}' - single-page-per-image")
        for img_path in img_paths:
            eff_array = get_img_array(img_path, IMG_SIZE, "efficientnet")
            vgg_array = get_img_array(img_path, IMG_SIZE, "vgg16")
            res_array = get_img_array(img_path, IMG_SIZE, "resnet50")
            eff_heatmap = make_gradcam_heatmap(eff_array, eff_model, eff_layer)
            vgg_heatmap = make_gradcam_heatmap(vgg_array, vgg_model, vgg_layer)
            res_heatmap = make_gradcam_heatmap(res_array, res_model, res_layer)
            display_and_save_gradcam(img_path, [eff_heatmap, vgg_heatmap, res_heatmap],
                                     ["EfficientNetB0","VGG16","ResNet50"], pdf)

    # One page per class
    for cls, img_paths in selected_images.items():
        print(f"Processing class '{cls}' - single-page-per-class")
        n_imgs = len(img_paths)
        fig, axes = plt.subplots(n_imgs, 4, figsize=(20, 5 * n_imgs))  # 4 cols: Original + 3 models
        if n_imgs == 1:
            axes = np.expand_dims(axes, axis=0)  # make 2D array shape (1,4)
        fig.suptitle(f"{cls} - {N_PER_CLASS} Random Images", fontsize=16, y=1.02)
        for row, img_path in enumerate(img_paths):
            eff_array = get_img_array(img_path, IMG_SIZE, "efficientnet")
            vgg_array = get_img_array(img_path, IMG_SIZE, "vgg16")
            res_array = get_img_array(img_path, IMG_SIZE, "resnet50")
            eff_heatmap = make_gradcam_heatmap(eff_array, eff_model, eff_layer)
            vgg_heatmap = make_gradcam_heatmap(vgg_array, vgg_model, vgg_layer)
            res_heatmap = make_gradcam_heatmap(res_array, res_model, res_layer)

            original = cv2.imread(img_path)
            original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
            original = cv2.resize(original, (224, 224))

            axes[row][0].imshow(original)
            axes[row][0].set_title("Original")
            axes[row][0].axis("off")

            for col, (heatmap, title) in enumerate(zip([eff_heatmap, vgg_heatmap, res_heatmap],
                                                       ["EfficientNetB0","VGG16","ResNet50"]), start=1):
                heatmap_resized = cv2.resize(heatmap, (224,224))
                heatmap_colored = cv2.applyColorMap(np.uint8(255*heatmap_resized), cv2.COLORMAP_JET)
                heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)
                superimposed_img = cv2.addWeighted(original, 0.6, heatmap_colored, 0.4, 0)
                axes[row][col].imshow(superimposed_img)
                axes[row][col].set_title(title)
                axes[row][col].axis("off")

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        pdf.savefig(fig, dpi=300)
        plt.show()
        plt.close(fig)

print(f"Grad-CAM results saved to {pdf_output}")


# 11.Save the model 

In [ ]:
# Save the model in Keras native format (.keras)
model.save('/kaggle/working/MangoModel.keras', overwrite=True)
model.save('/kaggle/working/MangoModel.h5', overwrite=True)
print("Model saved at /kaggle/working/MangoModel.keras. Download it from the sidebar.")